<a href="https://colab.research.google.com/github/dudaholandah/PIBIC/blob/main/Nutricao/Coordenadas_Paralelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!apt-get install poppler-utils
!apt-get update poppler-utils
!pip install pdf2image
!pip install unidecode
!pip install -U kaleido
!pip install scikit-learn-extra
# !pip install umap-learn[plot]
# !pip install trimap

In [5]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn import preprocessing
from pandas.core.frame import DataFrame
import plotly.express as px
import re
import plotly.graph_objects as go
import plotly
import pdf2image
import os

# Pre-Processing

In [6]:
data = pd.read_excel("TABLE_VEG.xlsx", sheet_name='Veganos')

In [9]:
data = data.drop(columns=['Porção', 'Kcal', 'Carboidratos', 'Açúcares', 'Proteínas', 'Gorduras Totais', 'Gorduras Saturadas', 'Fibras ', 'Sódio', 'B12', 'Cálcio', 'Zinco'])

In [10]:
data.head()

,Classificação,Nome do Produto,Ingredientes,Kcal.1,Carboidratos.1,Açúcares.1,Proteínas.1,Gorduras Totais.1,Gorduras Saturadas.1,Fibras .1,Sódio.1,B12.1,Cálcio.1,Zinco.1
0,CARNES 1,Hambúrguer Fazenda Futuro 2.0,"água, proteína texturizada de soja, proteína d...",246.25,12.50,0.0,13.75,16.25,5.25,3.500,595.0,0.0,0.0,0.0
1,CARNES 2,Carne Moída do Futuro,"água, proteína texturizada de soja, proteína d...",210.00,13.75,0.0,10.25,12.50,11.25,0.875,490.0,0.0,0.0,0.0
2,CARNES 3,Almôndega do Futuro,"água, proteína texturizada de soja, proteína i...",210.00,13.75,0.0,10.25,12.50,11.25,0.875,490.0,0.0,0.0,0.0
3,FRANGOS 3,Frango do Futuro,"água, proteína texturizada de soja, proteína i...",210.00,13.75,0.0,10.25,12.50,11.25,0.875,490.0,0.0,0.0,0.0
4,SUÍNOS 1,Linguiça do Futuro,"água, proteína texturizada de soja, proteína i...",278.00,15.40,0.0,11.40,19.00,7.80,0.400,588.0,0.0,0.0,0.0


In [13]:
for index, row in data.iterrows():
  label = data['Classificação'][index].rstrip().lstrip()
  data.at[index,'Classificação'] = label

print(data['Classificação'].value_counts())

LATICÍNIOS 1    78
LATICÍNIOS 3    45
CARNES 1        35
CARNES 3        18
FRANGOS 1       14
OVOS 1          14
FRANGOS 3       13
SUÍNOS 1        12
LATICÍNIOS 2    11
CARNES 2         9
PESCADOS 1       8
SUÍNOS 2         6
FRANGOS 2        5
PESCADOS 2       5
OVOS 2           2
Name: Classificação, dtype: int64


In [14]:
flag_dairy = []
flag_egg = []

for label in data['Classificação']:
    if "LATICÍNIOS" in label:
        flag_dairy.append(True)
    else:
        flag_dairy.append(False)

    if "OVOS" in label:
        flag_egg.append(True)
    else:
        flag_egg.append(False)  

data_dairy = data[flag_dairy]
data_egg = data[flag_egg]

In [18]:
data_egg['Classificação'] = data_egg['Classificação'].map({'OVOS 1': 1, 'OVOS 2': 2}) 
data_dairy['Classificação'] = data_dairy['Classificação'].map({'LATICÍNIOS 1': 1, 'LATICÍNIOS 2': 2, 'LATICÍNIOS 3': 3}) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
if not os.path.exists("images"):
    os.mkdir("images")

# Graphics

In [25]:
fig = px.parallel_coordinates(data_egg, 
                              color='Classificação', 
                              dimensions=['Kcal.1', 'Carboidratos.1', 'Proteínas.1', 'Gorduras Totais.1', 'Gorduras Saturadas.1', 'Fibras .1',
                                          ],
                              labels={'Classificação':'Classification', 'Kcal.1': "Energy (Kcal)", 'Carboidratos.1': 'Carbohydrate (g)', 'Proteínas.1': 'Protein (g)', 
                                      'Gorduras Totais.1': 'Total Fat (g)', 'Gorduras Saturadas.1': 'Saturated Fat (g)', 'Fibras .1': 'Dietary Fiber (g)',
                                      }, 
                              range_color=[0.5,2.5],
                              color_continuous_scale=[(0.0, px.colors.qualitative.Plotly[0]), (0.5, px.colors.qualitative.Plotly[0]),
                                                      (0.5, px.colors.qualitative.Plotly[1]), (1.0, px.colors.qualitative.Plotly[1]),
                              ],
                             )

fig.update_layout(coloraxis_colorbar=dict(
      tickvals=[1,2],
      ticktext=["Mayonnaise","Eggs"],
      lenmode="pixels", len=100,
))

fig.write_image('images/EGG.png')
fig.show()

In [26]:
fig = px.parallel_coordinates(data_dairy, 
                              color='Classificação', 
                              dimensions=['Kcal.1', 'Carboidratos.1', 'Proteínas.1', 'Gorduras Totais.1', 'Gorduras Saturadas.1', 'Fibras .1',
                                          ],
                              labels={'Classificação':'Classification', 'Kcal.1': "Energy (Kcal)", 'Carboidratos.1': 'Carbohydrate (g)', 'Proteínas.1': 'Protein (g)', 
                                      'Gorduras Totais.1': 'Total Fat (g)', 'Gorduras Saturadas.1': 'Saturated Fat (g)', 'Fibras .1': 'Dietary Fiber (g)',
                                      },
                              range_color=[0.8,3.2],
                              color_continuous_scale=[(0.0, px.colors.qualitative.Plotly[0]), (0.33, px.colors.qualitative.Plotly[0]),
                                                      (0.33, px.colors.qualitative.Plotly[1]), (0.66, px.colors.qualitative.Plotly[1]),
                                                      (0.66, px.colors.qualitative.Plotly[2]), (1.0, px.colors.qualitative.Plotly[2]),
                              ],
                             )

fig.update_layout(coloraxis_colorbar=dict(
      tickvals=[1,2,3],
      ticktext=["Beverages","Yogurts","Cheese"],
      lenmode="pixels", len=140,
))

fig.write_image('images/DAIRY.png')
fig.show()